<font size="6">Use network packets to finetune the model</font>

<font size="6">Step1: Preprocess the dataset</font>

In [1]:
import pandas as pd
import json
import os
import random
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np


In [4]:
prompt_template = """### Network packet information:
{information}

### The type of this netowork packet is:
"""

In [ ]:
filepath = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/iot23_combined.csv"
# df = pd.read_csv(filepath, nrows=1000)
df = pd.read_csv(filepath)
# df = df.drop_duplicates()
sampled_df = pd.DataFrame()  # Create an empty DataFrame，to store the filter dataset 
for label_value in ["Okiru", "DDoS"]: #,   "PartOfAHorizontalPortScan", "Benign", "C&C", "Okiru", "DDoS"
    type_subset = df[df['label'] == label_value].sample(n=3200, random_state=1)
    sampled_df = pd.concat([sampled_df, type_subset])


del sampled_df['Unnamed: 0']
X = sampled_df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = sampled_df['label']
data_dict = sampled_df.to_dict(orient='records')
data_dict
# sampled_df.to_csv('sampled_iot_data.csv', index=False)


In [ ]:
# data_unique = sampled_df.drop_duplicates()
# sampled_df

In [12]:
#modify the dataframe column name 
# new_sampled_df=sampled_df.rename(columns={"duration":"duration","orig_bytes": "payload_bytes_the_originator_sent", "resp_bytes": "payload_bytes_the_responder_sent","orig_pkts":"number_of_packets_that_the_originator_sent",
#                                           "orig_ip_bytes":"originator_ip_bytes","resp_pkts":"responder_pkts",
#                                           "resp_ip_bytes":"responder_ip_bytes","proto_icmp":"protocol_icmp",
#                                           "proto_tcp":"protocol_tcp","proto_udp":"protocol_udp",
#                                           "conn_state_OTH": "connection_state_OTH", "conn_state_REJ":"connection_state_REJ", 
#                                           "conn_state_RSTO":"connection_state_RSTO", "conn_state_RSTOS0":"connection_state_RSTOS0", 
#                                           "conn_state_RSTR":"connection_state_RSTR", "conn_state_RSTRH":"connection_state_RSTRH", 
#                                           "conn_state_S0":"connection_state_S0", "conn_state_S1":"connection_state_S1", 
#                                           "conn_state_S2":"connection_state_S2", "conn_state_S3":"connection_state_S3", 
#                                           "conn_state_SF":"connection_state_SF", "conn_state_SH":"connection_state_SH", 
#                                           "conn_state_SHR":"connection_state_SHR"
#                                           })
new_sampled_df2=sampled_df.rename(columns={"duration":"duration","orig_bytes": "payload bytes the originator sent", 
                                           "resp_bytes": "payload bytes the responder sent",
                                           "missed_bytes": "missed bytes",
                                           "orig_pkts":"number of packets that the originator sent",
                                          "orig_ip_bytes":"originator ip bytes","resp_pkts":"responder pkts",
                                          "resp_ip_bytes":"responder ip bytes","proto_icmp":"protocol icmp",
                                          "proto_tcp":"protocol tcp","proto_udp":"protocol udp",
                                          "conn_state_OTH": "connection state OTH", "conn_state_REJ":"connection state REJ", 
                                          "conn_state_RSTO":"connection state RSTO", "conn_state_RSTOS0":"connection state RSTOS0", 
                                          "conn_state_RSTR":"connection state RSTR", "conn_state_RSTRH":"connection state RSTRH", 
                                          "conn_state_S0":"connection state S0", "conn_state_S1":"connection state S1", 
                                          "conn_state_S2":"connection state S2", "conn_state_S3":"connection state S3", 
                                          "conn_state_SF":"connection state SF", "conn_state_SH":"connection state SH", 
                                          "conn_state_SHR":"connection state SHR"
                                          })



new_sampled_df2.to_csv("/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/Experiment8/new_sampled2_df_6400_Okiru_DDoS_original.csv",index=False)
colmn=pd.read_csv("/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/Experiment8/new_sampled2_df_6000_Okiru_DDoS_original.csv")

print([column for column in colmn])


['duration', 'payload bytes the originator sent', 'payload bytes the responder sent', 'missed bytes', 'number of packets that the originator sent', 'originator ip bytes', 'responder pkts', 'responder ip bytes', 'label', 'protocol icmp', 'protocol tcp', 'protocol udp', 'connection state OTH', 'connection state REJ', 'connection state RSTO', 'connection state RSTOS0', 'connection state RSTR', 'connection state RSTRH', 'connection state S0', 'connection state S1', 'connection state S2', 'connection state S3', 'connection state SF', 'connection state SH', 'connection state SHR']


In [ ]:
#直接读取csv文件
filepath = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/experiment7_combined_data.csv"
sampled_df = pd.read_csv(filepath)
sampled_df

In [8]:
data_unique = sampled_df.drop_duplicates()
data_unique['label'].value_counts()

PartOfAHorizontalPortScan    1000
Name: label, dtype: int64

<font size="5">Data prepare</font>

In [14]:
#Data prepare,合并train data，取得validation和test data
filepath = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/Experiment8/experiment8_16000_combined_data.csv"
sampled_df = pd.read_csv(filepath) 
df_shuffled = sampled_df.sample(frac=1)  #打乱这个数据
folderpath="/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/Experiment8"

# 划分数据集
train_data = df_shuffled[:5000]
validation_data = df_shuffled[5000:6000]
test_data = df_shuffled[6000:]

# 保存 train data 和 validation data
train_data.to_csv(folderpath+'/Experiment8_5000_train_data.csv', index=False)
validation_data.to_csv(folderpath+'/Experiment8_1000validation_data.csv', index=False)

# 分割 test data 并保存
test_data_splits = np.array_split(test_data, 10)
for i, split in enumerate(test_data_splits):
    split.to_csv(folderpath+f'/Experiment8_test_data_{i+1}.csv', index=False)





In [2]:
#transfer csv file to jsonl

#生成josn list
def csv_transfer_jsonal(sampled_data_csv,jsonal_file_path):
    sampled_df_data = pd.read_csv(sampled_data_csv)
    json_list_data = []
    for index, row in sampled_df_data.iterrows():
        # X = row[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']] #original label
        # X = row[['duration', 'payload bytes_the_originator_sent', 'payload_bytes_the_responder_sent', 'missed_bytes', 'number_of_packets_that_the_originator_sent', 'originator_ip_bytes', 'responder_pkts', 'responder_ip_bytes', 'protocol_icmp', 'protocol_tcp', 'protocol_udp', 'connection_state_OTH', 'connection_state_REJ', 'connection_state_RSTO', 'connection_state_RSTOS0', 'connection_state_RSTR', 'connection_state_RSTRH', 'connection_state_S0', 'connection_state_S1', 'connection_state_S2', 'connection_state_S3', 'connection_state_SF', 'connection_state_SH', 'connection_state_SHR']] #My label
        X = row[['duration', 'payload bytes the originator sent', 'payload bytes the responder sent', 'missed bytes', 'number of packets that the originator sent', 'originator ip bytes', 'responder pkts', 'responder ip bytes', 'protocol icmp', 'protocol tcp', 'protocol udp', 'connection state OTH', 'connection state REJ', 'connection state RSTO', 'connection state RSTOS0', 'connection state RSTR', 'connection state RSTRH', 'connection state S0', 'connection state S1', 'connection state S2', 'connection state S3', 'connection state SF', 'connection state SH', 'connection state SHR']] #My label
        Y = row['label']
        X_dict = X.to_dict()    

        json_object_gpt_turbo = {"messages": [{"role": "system", "content": "You are a good network packet analyzer\ Please analyze the network packet information provided by the user and determine its type\ The possible types are 'PartOfAHorizontalPortScan', 'Okiru', 'Benign', 'DDoS', 'C&C'\ Provide only the most likely type from these options as final output\ Only output this type and nothing else\ Double-check your answer to ensure accuracy before outputting"}, {"role": "user", "content": prompt_template.format(information=str(X_dict))}, {"role": "assistant", "content":Y }]}
        json_object_babbage= {"prompt": prompt_template.format(information=str(X_dict)), "completion": Y}
        json_list_data.append(json_object_gpt_turbo)
    # new method: store train and test data to jsonal file
    # Write train data to file
    with open(jsonal_file_path, "w") as train_file:
        for json_object in json_list_data:
            line = json.dumps(json_object) + "\n"
            train_file.write(line)
    

In [12]:
csv_transfer_jsonal("/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/Experiment8/Experiment8_test_data_10.csv",
                    "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/Experiment8/Experiment8_test_data_10.jsonl")

In [5]:
#normalize
# columns_to_normalize = ['duration', 'payload bytes the originator sent', 'payload bytes the responder sent', 'missed bytes', 'number of packets that the originator sent', 'originator ip bytes', 'responder pkts', 'responder ip bytes', 'protocol icmp', 'protocol tcp', 'protocol udp', 'connection state OTH', 'connection state REJ', 'connection state RSTO', 'connection state RSTOS0', 'connection state RSTR', 'connection state RSTRH', 'connection state S0', 'connection state S1', 'connection state S2', 'connection state S3', 'connection state SF', 'connection state SH', 'connection state SHR'] #My label
# scaler = MinMaxScaler()
# sampled_df[columns_to_normalize] = scaler.fit_transform(sampled_df[columns_to_normalize])
# sampled_df

,duration,payload bytes the originator sent,payload bytes the responder sent,missed bytes,number of packets that the originator sent,originator ip bytes,responder pkts,responder ip bytes,label,protocol icmp,...,connection state RSTOS0,connection state RSTR,connection state RSTRH,connection state S0,connection state S1,connection state S2,connection state S3,connection state SF,connection state SH,connection state SHR
0,0.000044,0.000609,0.000255,0.0,0.001342,0.001482,0.000000,0.000000,PartOfAHorizontalPortScan,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000044,0.000609,0.000255,0.0,0.001342,0.000988,0.000000,0.000000,PartOfAHorizontalPortScan,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.002685,0.002965,0.000000,0.000000,PartOfAHorizontalPortScan,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.0,0.005369,0.003953,0.000000,0.000000,PartOfAHorizontalPortScan,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000044,0.000609,0.000255,0.0,0.001342,0.000988,0.000000,0.000000,PartOfAHorizontalPortScan,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.000183,0.000609,0.000255,0.0,0.004027,0.004447,0.000000,0.000000,C&C,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,0.000044,0.000609,0.000255,0.0,0.001342,0.001482,0.000000,0.000000,C&C,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,0.000089,0.037150,0.031855,0.0,0.006711,0.008104,0.002751,0.004626,C&C,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,0.000184,0.000609,0.000255,0.0,0.004027,0.004447,0.000000,0.000000,C&C,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# #Old method to divide data,
# # Create a list to store the converted data
# json_list = []

# for index, row in sampled_df.iterrows():
#     # X = row[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']] #original label
#     # X = row[['duration', 'payload bytes_the_originator_sent', 'payload_bytes_the_responder_sent', 'missed_bytes', 'number_of_packets_that_the_originator_sent', 'originator_ip_bytes', 'responder_pkts', 'responder_ip_bytes', 'protocol_icmp', 'protocol_tcp', 'protocol_udp', 'connection_state_OTH', 'connection_state_REJ', 'connection_state_RSTO', 'connection_state_RSTOS0', 'connection_state_RSTR', 'connection_state_RSTRH', 'connection_state_S0', 'connection_state_S1', 'connection_state_S2', 'connection_state_S3', 'connection_state_SF', 'connection_state_SH', 'connection_state_SHR']] #My label
#     X = row[['duration', 'payload bytes the originator sent', 'payload bytes the responder sent', 'missed bytes', 'number of packets that the originator sent', 'originator ip bytes', 'responder pkts', 'responder ip bytes', 'protocol icmp', 'protocol tcp', 'protocol udp', 'connection state OTH', 'connection state REJ', 'connection state RSTO', 'connection state RSTOS0', 'connection state RSTR', 'connection state RSTRH', 'connection state S0', 'connection state S1', 'connection state S2', 'connection state S3', 'connection state SF', 'connection state SH', 'connection state SHR']] #My label
#     Y = row['label']
#     X_dict = X.to_dict()

    

#     # print(str(X_dict))
#     # print(Y)
#     # print(type(Y))

#     json_object_gpt_turbo = {"messages": [{"role": "system", "content": "You are a good network packet analyzer\ Please analyze the network packet information provided by the user and determine its type\ The possible types are 'PartOfAHorizontalPortScan', 'Okiru', 'Benign', 'DDoS', 'C&C'\ Provide only the most likely type from these options as final output\ Only output this type and nothing else"}, {"role": "user", "content": prompt_template.format(information=str(X_dict))}, {"role": "assistant", "content":Y }]}
#     json_object_babbage= {"prompt": prompt_template.format(information=str(X_dict)), "completion": Y}
#     json_list.append(json_object_gpt_turbo)
#     # print(json_list)
    
    

In [15]:
# # Old method to divide data, shuffle the jsonlist. Write the converted data to a JSON file
# # output_file_path = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/dataset_babbage_5000.json"  # Replace with the desired output file path


# # Calculate the split index
# split_index = int(0.8 * len(json_list))

# # Split the list into two parts randomly 
# random.shuffle(json_list)   #打乱列表使得里面的数据随机
# train_data = json_list[:split_index]
# test_data = json_list[split_index:]

# # Calculate the train dataset number and test dataset number
# # train_data_number = int(0.8 * len(json_list))
# # # Split the list into two parts randomly
# # train_data= random.sample(json_list, train_data_number)
# # # test_data = [item for item in json_list if item not in train_data]
# # test_data = set(json_list) - set(train_data)


# # File paths for train and test data
# train_file_path = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/new_label2_dataset_gpt_turbo_5000_train.jsonl"
# test_file_path = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/new_label2_dataset_gpt_turbo_5000_test.jsonl"
# # Write train data to file
# with open(train_file_path, "w") as train_file:
#     for json_object in train_data:
#         line = json.dumps(json_object) + "\n"
#         train_file.write(line)

# # Write test data to file
# with open(test_file_path, "w") as test_file:
#     for json_object in test_data:
#         line = json.dumps(json_object) + "\n"
#         test_file.write(line)

# print(
#     "JSON objects have been split (80% \and 20%) and written to dataset_5000_train.jsonl and dataset_5000_test.jsonl"
# )

JSON objects have been split (80% nd 20%) and written to dataset_5000_train.jsonl and dataset_5000_test.jsonl


In [13]:
#New method to divide data, keep the train data same

df_shuffled = sampled_df.sample(frac=1)   #Shuffle this data set, frac=1 means keep all data (打乱这个数据集，frac=1表示保留所有数据)
split_ratio = 0.8            # 80% of the data is used for training and 20% of the data is used for testing (80%的数据用于训练，20%的数据用于测试)
split_index = int(split_ratio * len(df_shuffled))

sampled_df_train_data = df_shuffled[:split_index]
sampled_df_test_data = df_shuffled[split_index:]

#store the train data and test data to diffent csv file
sampled_df_train_data.to_csv("/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/Experiment7_5000_train.csv",index=False)
sampled_df_test_data.to_csv("/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/Experiment7_5000_test.csv",index=False)


In [16]:
#new method: create json list fot train and test data
json_list_train_data = []
json_list_test_data = []

#生成train data的josn list
for index, row in sampled_df_train_data.iterrows():
    # X = row[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']] #original label
    # X = row[['duration', 'payload bytes_the_originator_sent', 'payload_bytes_the_responder_sent', 'missed_bytes', 'number_of_packets_that_the_originator_sent', 'originator_ip_bytes', 'responder_pkts', 'responder_ip_bytes', 'protocol_icmp', 'protocol_tcp', 'protocol_udp', 'connection_state_OTH', 'connection_state_REJ', 'connection_state_RSTO', 'connection_state_RSTOS0', 'connection_state_RSTR', 'connection_state_RSTRH', 'connection_state_S0', 'connection_state_S1', 'connection_state_S2', 'connection_state_S3', 'connection_state_SF', 'connection_state_SH', 'connection_state_SHR']] #My label
    X = row[['duration', 'payload bytes the originator sent', 'payload bytes the responder sent', 'missed bytes', 'number of packets that the originator sent', 'originator ip bytes', 'responder pkts', 'responder ip bytes', 'protocol icmp', 'protocol tcp', 'protocol udp', 'connection state OTH', 'connection state REJ', 'connection state RSTO', 'connection state RSTOS0', 'connection state RSTR', 'connection state RSTRH', 'connection state S0', 'connection state S1', 'connection state S2', 'connection state S3', 'connection state SF', 'connection state SH', 'connection state SHR']] #My label
    Y = row['label']
    X_dict = X.to_dict()    

    json_object_gpt_turbo = {"messages": [{"role": "system", "content": "You are a good network packet analyzer\ Please analyze the network packet information provided by the user and determine its type\ The possible types are 'PartOfAHorizontalPortScan', 'Okiru', 'Benign', 'DDoS', 'C&C'\ Provide only the most likely type from these options as final output\ Only output this type and nothing else\ Double-check your answer to ensure accuracy before outputting"}, {"role": "user", "content": prompt_template.format(information=str(X_dict))}, {"role": "assistant", "content":Y }]}
    json_object_babbage= {"prompt": prompt_template.format(information=str(X_dict)), "completion": Y}
    json_list_train_data.append(json_object_gpt_turbo)
    # print(json_list)

#生成test data的josn list
for index, row in sampled_df_test_data.iterrows():
    # X = row[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']] #original label
    # X = row[['duration', 'payload bytes_the_originator_sent', 'payload_bytes_the_responder_sent', 'missed_bytes', 'number_of_packets_that_the_originator_sent', 'originator_ip_bytes', 'responder_pkts', 'responder_ip_bytes', 'protocol_icmp', 'protocol_tcp', 'protocol_udp', 'connection_state_OTH', 'connection_state_REJ', 'connection_state_RSTO', 'connection_state_RSTOS0', 'connection_state_RSTR', 'connection_state_RSTRH', 'connection_state_S0', 'connection_state_S1', 'connection_state_S2', 'connection_state_S3', 'connection_state_SF', 'connection_state_SH', 'connection_state_SHR']] #My label
    X = row[['duration', 'payload bytes the originator sent', 'payload bytes the responder sent', 'missed bytes', 'number of packets that the originator sent', 'originator ip bytes', 'responder pkts', 'responder ip bytes', 'protocol icmp', 'protocol tcp', 'protocol udp', 'connection state OTH', 'connection state REJ', 'connection state RSTO', 'connection state RSTOS0', 'connection state RSTR', 'connection state RSTRH', 'connection state S0', 'connection state S1', 'connection state S2', 'connection state S3', 'connection state SF', 'connection state SH', 'connection state SHR']] #My label
    Y = row['label']
    X_dict = X.to_dict()    

    json_object_gpt_turbo = {"messages": [{"role": "system", "content": "You are a good network packet analyzer\ Please analyze the network packet information provided by the user and determine its type\ The possible types are 'PartOfAHorizontalPortScan', 'Okiru', 'Benign', 'DDoS', 'C&C'\ Provide only the most likely type from these options as final output\ Only output this type and nothing else\ Double-check your answer to ensure accuracy before outputting "}, {"role": "user", "content": prompt_template.format(information=str(X_dict))}, {"role": "assistant", "content":Y }]}
    json_object_babbage= {"prompt": prompt_template.format(information=str(X_dict)), "completion": Y}
    json_list_test_data.append(json_object_gpt_turbo)

In [17]:
# new method: store train and test data to jsonal file
# File paths for train and test data
train_file_path = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/new_label2_dataset_gpt_turbo_10000_train_Experiment5.jsonl"
test_file_path = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/new_label2_dataset_gpt_turbo_10000_test_Experiment5.jsonl"
# Write train data to file
with open(train_file_path, "w") as train_file:
    for json_object in json_list_train_data:
        line = json.dumps(json_object) + "\n"
        train_file.write(line)

# Write test data to file
with open(test_file_path, "w") as test_file:
    for json_object in json_list_test_data:
        line = json.dumps(json_object) + "\n"
        test_file.write(line)

print(
    "JSON objects have been split (80% \and 20%) and written to dataset_5000_train.jsonl and dataset_5000_test.jsonl"
)

JSON objects have been split (80% nd 20%) and written to dataset_5000_train.jsonl and dataset_5000_test.jsonl


<font size="6">Step2.1: Upload the training dataset to openAI</font>

In [18]:
from openai import OpenAI
client = OpenAI(api_key='**********')


# Path to training data
training_dataset = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/new_label2_dataset_gpt_turbo_10000_train_Experiment5.jsonl"
test_dataset= "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/new_label2_dataset_gpt_turbo_10000_test_Experiment5.jsonl"
# Upload train file to openai
client.files.create(
  # file=open(test_dataset, "rb"),
  file=open(training_dataset, "rb"),
  purpose="fine-tune"
)

# Upload test file to openai
client.files.create(
  # file=open(test_dataset, "rb"),
  file=open(test_dataset, "rb"),
  purpose="fine-tune"
)


# 1. FileObject(id='file-cGUYmAcGEA1yald9ybSFDBzX', bytes=159095, created_at=1706025370, filename='dataset_1000_test.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
#babbage:FileObject(id='file-OwXvvH4pDQ5ERzZDrASTWWFF', bytes=133495, created_at=1706030310, filename='dataset_babbage_1000_test.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
#babbage test:file-Digcrly10euFMAj9VdYRByHC



FileObject(id='file-C8XS5VX5bsZXot93JtwS6cBy', bytes=2645762, created_at=1713965965, filename='new_label2_dataset_gpt_turbo_10000_test_Experiment5.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

<font size="6">Step2.2: Create a finetone job</font>

In [40]:
from openai import OpenAI
client = OpenAI(api_key='**********')


# check file id: print(openai.File.list())
file_id = "**********"

client.fine_tuning.jobs.create(
  training_file=file_id, 
  model="babbage-002"
)

#chatgpt3.5: FineTuningJob(id='ftjob-PRMI4gtCcxXgXfHhU0QktlEv', created_at=1706025417, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-gS5nata0XGKHO0BeClpnTQny', result_files=[], status='validating_files', trained_tokens=None, training_file='file-cGUYmAcGEA1yald9ybSFDBzX', validation_file=None)
#babbage: FineTuningJob(id='ftjob-Qq3oFe9gf1cQCcBEvwy7LVkE', created_at=1706030370, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-gS5nata0XGKHO0BeClpnTQny', result_files=[], status='validating_files', trained_tokens=None, training_file='file-OwXvvH4pDQ5ERzZDrASTWWFF', validation_file=None)


FineTuningJob(id='ftjob-Qq3oFe9gf1cQCcBEvwy7LVkE', created_at=1706030370, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-gS5nata0XGKHO0BeClpnTQny', result_files=[], status='validating_files', trained_tokens=None, training_file='file-OwXvvH4pDQ5ERzZDrASTWWFF', validation_file=None)

<font size="6">Step2.3: Check fine-tune job info</font>

In [24]:
from openai import OpenAI
client = OpenAI(api_key='************')
# List fine-tuning jobs
client.fine_tuning.jobs.list()

# Retrieve the state of a fine-tune
# client.fine_tuning.jobs.retrieve("ftjob-Qq3oFe9gf1cQCcBEvwy7LVkE")

FineTuningJob(id='ftjob-GJx5k8DUVM2kGpe4uxUi5Pgy', created_at=1706046133, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='babbage-002', object='fine_tuning.job', organization_id='org-gS5nata0XGKHO0BeClpnTQny', result_files=[], status='cancelled', trained_tokens=None, training_file='file-OwXvvH4pDQ5ERzZDrASTWWFF', validation_file='file-Digcrly10euFMAj9VdYRByHC')

<font size="6">Step3: Try the fine-tuned model</font>

In [5]:
test_information= "{'duration': 2.999077, 'orig_bytes': 0, 'resp_bytes': 0, 'missed_bytes': 0.0, 'orig_pkts': 3.0, 'orig_ip_bytes': 180.0, 'resp_pkts': 0.0, 'resp_ip_bytes': 0.0, 'proto_icmp': False, 'proto_tcp': True, 'proto_udp': False, 'conn_state_OTH': False, 'conn_state_REJ': False, 'conn_state_RSTO': False, 'conn_state_RSTOS0': False, 'conn_state_RSTR': False, 'conn_state_RSTRH': False, 'conn_state_S0': True, 'conn_state_S1': False, 'conn_state_S2': False, 'conn_state_S3': False, 'conn_state_SF': False, 'conn_state_SH': False, 'conn_state_SHR': False}" #PartOfAHorizontalPortScan
test_information2="{'duration': 0.0, 'orig_bytes': 0, 'resp_bytes': 0, 'missed_bytes': 0.0, 'orig_pkts': 1.0, 'orig_ip_bytes': 40.0, 'resp_pkts': 0.0, 'resp_ip_bytes': 0.0, 'proto_icmp': False, 'proto_tcp': False, 'proto_udp': True, 'conn_state_OTH': False, 'conn_state_REJ': False, 'conn_state_RSTO': False, 'conn_state_RSTOS0': False, 'conn_state_RSTR': False, 'conn_state_RSTRH': False, 'conn_state_S0': True, 'conn_state_S1': False, 'conn_state_S2': False, 'conn_state_S3': False, 'conn_state_SF': False, 'conn_state_SH': False, 'conn_state_SHR': False}" #benign
test_information3= "{'duration': 0.0, 'orig_bytes': 0, 'resp_bytes': 0, 'missed_bytes': 0.0, 'orig_pkts': 1.0, 'orig_ip_bytes': 60.0, 'resp_pkts': 0.0, 'resp_ip_bytes': 0.0, 'proto_icmp': False, 'proto_tcp': True, 'proto_udp': False, 'conn_state_OTH': False, 'conn_state_REJ': False, 'conn_state_RSTO': False, 'conn_state_RSTOS0': False, 'conn_state_RSTR': False, 'conn_state_RSTRH': False, 'conn_state_S0': True, 'conn_state_S1': False, 'conn_state_S2': False, 'conn_state_S3': False, 'conn_state_SF': False, 'conn_state_SH': False, 'conn_state_SHR': False}" #PartOfAHorizontalPortScan
test_information4= "{'duration': 0.0, 'orig_bytes': 0, 'resp_bytes': 0, 'missed_bytes': 0.0, 'orig_pkts': 1.0, 'orig_ip_bytes': 40.0, 'resp_pkts': 0.0, 'resp_ip_bytes': 0.0, 'proto_icmp': False, 'proto_tcp': False, 'proto_udp': True, 'conn_state_OTH': False, 'conn_state_REJ': False, 'conn_state_RSTO': False, 'conn_state_RSTOS0': False, 'conn_state_RSTR': False, 'conn_state_RSTRH': False, 'conn_state_S0': True, 'conn_state_S1': False, 'conn_state_S2': False, 'conn_state_S3': False, 'conn_state_SF': False, 'conn_state_SH': False, 'conn_state_SHR': False}" #benign
test_information5="{'duration': 2.998829, 'orig_bytes': 0, 'resp_bytes': 0, 'missed_bytes': 0.0, 'orig_pkts': 3.0, 'orig_ip_bytes': 180.0, 'resp_pkts': 0.0, 'resp_ip_bytes': 0.0, 'proto_icmp': False, 'proto_tcp': True, 'proto_udp': False, 'conn_state_OTH': False, 'conn_state_REJ': False, 'conn_state_RSTO': False, 'conn_state_RSTOS0': False, 'conn_state_RSTR': False, 'conn_state_RSTRH': False, 'conn_state_S0': True, 'conn_state_S1': False, 'conn_state_S2': False, 'conn_state_S3': False, 'conn_state_SF': False, 'conn_state_SH': False, 'conn_state_SHR': False}" #benign


In [ ]:
#test finntuned gpt3.5 model
from openai import OpenAI

client = OpenAI(api_key='***********')


response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal:5k-newlabel:9HCxFiMg",  #finetune model name
  messages=[
    {"role": "system", "content": "You are a good network package analyzer\ Please analyze the network packet information provided by the user and determine its type\ The possible types are 'PartOfAHorizontalPortScan', 'Okiru', 'Benign', 'DDoS', 'C&C'\ Provide only the most likely type from these options as final output\ Only output this type and nothing else"},
    {"role": "user", "content": prompt_template.format(information=test_information)}
  ]   
)

print(response.choices[0].message.content)

In [ ]:
#test funetune babbage model
from openai import OpenAI

client = OpenAI(api_key='**********')

completion = client.completions.create(model='ft:gpt-3.5-turbo-0125:personal:turbo0125-1000:8zRrOLmx', prompt=prompt_template.format(information=str(test_information2)))
print(completion.choices[0].text)
# print(dict(completion).get('usage'))
# print(completion.model_dump_json(indent=2))



<font size="6">Step4: Evaluate the fine-tuned model</font>

In [3]:
#evaluate the baggage model
import json
import csv
from openai import OpenAI
def evaluate_finetuned_model(test_file,result_file):
    client = OpenAI(api_key='*************')
    finetuned_model_id="*************"
    total_rows = 0
    correct_predicition=0
    error_predicition=0
    
    # completion = client.completions.create(model='ft:babbage-002:personal::8p5AswsM', prompt=prompt_template.format(information=str(test_information4)))
    with open(result_file, 'w', encoding='utf-8') as csvfile:
        fieldnames = ['original_label', 'model_prediction', 'whether_first_string_of_output_match_label']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        with open(test_file, 'r') as file:
            for line in file:
                total_rows+=1
                # 解析json数据
                data = json.loads(line)
                # 输出prompt和completion
                network_information = data['prompt']   
                label = data['completion']
                #用finetuned model 进行分析
                completion = client.completions.create(model=finetuned_model_id, prompt=prompt_template.format(information=str(network_information)))
                model_prediction=completion.choices[0].text     #记录下模型的completion的text    
                #判断output中是否有真实label
                # if(label in model_prediction):
                #     correct_predicition+=1
                # else:
                #     error_predicition+=1
                #判断output是否以label开头
                if model_prediction.startswith(label):
                    whether_first_string_of_output_match_label=True
                    correct_predicition+=1
                else:
                    whether_first_string_of_output_match_label=False
                    error_predicition+=1

                writer.writerow({
                    'original_label': label,
                    
                    'model_prediction': model_prediction,
                    'whether_first_string_of_output_match_label':whether_first_string_of_output_match_label
                })
    accurency=correct_predicition/total_rows
    print(f"The correct prediction number is {correct_predicition}")
    print(f"The error prediction number is {error_predicition}")
    print(f"The accurency of this model is {accurency}")


        


In [ ]:
test_file = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/new_label_dataset_babbage_10000_test.jsonl"
result_file = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Result_Files/new_label_result_davinvi_10000.csv"
evaluate_finetuned_model(test_file,result_file)

In [2]:
#evaluate gpt3.5 model

import json
import csv
from openai import OpenAI
def evaluate_finetuned_gpt_model(test_file,result_file):
    client = OpenAI(api_key='*************')
    finetuned_model_id="*************"
    base_model= "gpt-3.5-turbo"
    total_rows = 0
    correct_predicition=0
    error_predicition=0
    y_true=[]
    y_pred=[]
    
    # completion = client.completions.create(model='ft:babbage-002:personal::8p5AswsM', prompt=prompt_template.format(information=str(test_information4)))
    with open(result_file, 'w', encoding='utf-8') as csvfile:
        fieldnames = ['original_label', 'model_prediction', 'whether_first_string_of_output_match_label']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        with open(test_file, 'r') as file:
            for line in file:
                total_rows+=1
                # 解析json数据
                data = json.loads(line)
                # 输出改行的message中的user和assistant的content
                user_content = None
                assistant_content = None
                for message in data['messages']:
                    role = message['role']
                    content = message['content']
                    # 根据role提取对应的内容
                    if role == 'user':
                        user_content = content
                    elif role == 'assistant':
                        assistant_content = content
                
                #用finetuned model 进行分析
                response = client.chat.completions.create(
                    model=base_model,  #finetune model name
                    temperature=0,
                    messages=[
                        {"role": "system", "content": "You are a good network package analyzer\ Please analyze the network packet information provided by the user and determine its type\ The possible types are 'PartOfAHorizontalPortScan', 'Okiru', 'Benign', 'DDoS', 'C&C'\ Provide only the most likely type from these options as final output\ Only output this type and nothing else"},
                        {"role": "user", "content": prompt_template.format(information=user_content)}
                    ]   
                    ) 
                model_prediction=response.choices[0].message.content     #记录下模型的response的content    
                #判断output中是否有真实label
                # if(label in model_prediction):
                #     correct_predicition+=1
                # else:
                #     error_predicition+=1
                #判断output是否以label开头
                if model_prediction.startswith(assistant_content):   #判断finetuned后的模型的输出是否以原本正确的label开头
                    whether_first_string_of_output_match_label=True
                    correct_predicition+=1
                else:
                    whether_first_string_of_output_match_label=False
                    error_predicition+=1
                y_true.append(assistant_content)
                y_pred.append(model_prediction)
                writer.writerow({
                    'original_label': assistant_content,
                    
                    'model_prediction': model_prediction,
                    'whether_first_string_of_output_match_label':whether_first_string_of_output_match_label
                })
    accurency=correct_predicition/total_rows
    confusion_mat = confusion_matrix(y_true, y_pred)
    
    report = classification_report(y_true, y_pred, digits=3) 
    print(f"The correct prediction number is {correct_predicition}")
    print(f"The error prediction number is {error_predicition}")
    print(f"The accurency of this model is {accurency}")
    print("Confusion Matrix:")
    print(confusion_mat)
    print("Classifiction Report :")
    print(report)


In [5]:
test_file = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/Experiment8/Experiment8_test_data_10.jsonl"
result_file = "/home/hzhou/LLM_Encrypted_Traffic_Analysis/Anomaly-Detection-IoT23-main/Data Preprocessing/Experiment8/result/Experiment8_basemodel_3.5_result10.csv"
evaluate_finetuned_gpt_model(test_file,result_file)

The correct prediction number is 228
The error prediction number is 772
The accurency of this model is 0.228
Confusion Matrix:
[[ 45   2   3   0 154]
 [  1   0  33   0 167]
 [107   0   0   0  92]
 [  0   0   0   0 198]
 [ 10   3   2   0 183]]
Classifiction Report :
                           precision    recall  f1-score   support

                   Benign      0.276     0.221     0.245       204
                      C&C      0.000     0.000     0.000       201
                     DDoS      0.000     0.000     0.000       199
                    Okiru      0.000     0.000     0.000       198
PartOfAHorizontalPortScan      0.230     0.924     0.369       198

                 accuracy                          0.228      1000
                macro avg      0.101     0.229     0.123      1000
             weighted avg      0.102     0.228     0.123      1000



/home/hzhou/anaconda3/envs/GPU3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hzhou/anaconda3/envs/GPU3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hzhou/anaconda3/envs/GPU3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
